# Make new csv with relabelled data

Reads in a results.csv and create a full dataset with extra columns
Oct 21, 2020

In [3]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import subprocess as sp
import pickle
#import ipywidgets as widgets
import time

import pandas as pd

In [81]:
fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/autoscan_features.3.csv'
df_full=pd.read_csv(fname,sep=',',comment='#')
df_full.head()

,ID,OBJECT_TYPE,AMP,A_IMAGE,A_REF,BAND,B_IMAGE,B_REF,CCDID,COLMEDS,...,N3SIG3,N3SIG3SHIFT,N3SIG5,N3SIG5SHIFT,NN_DIST_RENORM,NUMNEGRN,SCALE,SNR,SPREADERR_MODEL,SPREAD_MODEL
0,10742010,0,0.808323,1.508,2.650069,i,0.950,1.899501,10,0.112077,...,0,-8,0,-9,0.674934,22,2.024122,7.722346,0.004629,-0.003718
1,8828139,0,0.975141,1.430,NaN,i,1.319,NaN,10,0.072256,...,0,5,0,5,NaN,21,1.379283,25.582918,0.002420,-0.000578
2,8229889,0,0.877440,0.505,NaN,z,0.494,NaN,57,0.106124,...,0,1,0,1,NaN,20,0.950069,3.914192,0.008863,-0.020044
3,11033218,0,0.681179,0.762,3.008957,i,0.449,2.855881,1,0.139071,...,0,-14,2,-27,1.240574,18,1.749488,5.068382,0.008041,0.000246
4,8341858,0,0.645906,1.205,1.352224,i,0.982,1.063333,30,0.123302,...,0,4,0,4,2.970234,16,1.930613,7.372579,0.005606,0.001771


In [82]:
df2=pd.read_csv('results.csv')
df2.head()

,ID,old_label,changed_label,final_label
0,10593439,1,0,0
1,8463672,1,0,0
2,11326222,1,0,0
3,8093642,1,0,0
4,9585364,0,1,1


In [83]:
## Add extra columns
df_full['old_OBJECT_TYPE']=df_full['OBJECT_TYPE']
df_full['check_relabel']=np.zeros(df_full.shape[0],dtype=bool) ## Whether checked for relabelling. Not necessarily modified
print(df_full.shape)

(898963, 42)


In [84]:
id_list=df2.ID.values

df=df_full.copy()
for i,j in df[df.ID.isin(id_list)].iterrows():
    print(i)
#     print(i,j.relabelled)
    iD=j.ID
    new_label=df2[df2.ID==iD]['final_label'].values[0]
    df.loc[i,'check_relabel']=True
    df.loc[i,'OBJECT_TYPE']=new_label
    

22229
81074
114086
174232
224960
263218
327614
327843
401976
430110
512022
569075
678136
700639
725571
772528
791039
852670
884118
888966


In [91]:
df[df.old_OBJECT_TYPE!=df.OBJECT_TYPE]

,ID,OBJECT_TYPE,AMP,A_IMAGE,A_REF,BAND,B_IMAGE,B_REF,CCDID,COLMEDS,...,N3SIG5,N3SIG5SHIFT,NN_DIST_RENORM,NUMNEGRN,SCALE,SNR,SPREADERR_MODEL,SPREAD_MODEL,old_OBJECT_TYPE,check_relabel
22229,10300511,1,0.663420,1.079,2.537148,i,0.806,2.240999,31,0.146982,...,0,-25,0.841531,18,1.619514,6.914831,0.006259,-0.003262,0,True
81074,7858258,0,0.498107,2.057,1.149679,g,1.105,1.112527,56,0.097982,...,0,4,2.596035,14,2.631671,7.260512,0.003827,0.007316,1,True
114086,11195371,1,0.883662,1.425,3.396699,r,1.326,2.990418,15,0.069619,...,2,-32,0.731977,21,1.605894,13.715366,0.003736,-0.001215,0,True
224960,7811556,0,0.722201,0.505,2.175558,g,0.494,1.940895,46,0.094874,...,0,-16,0.186126,21,1.364391,4.235127,0.008123,-0.016148,1,True
263218,1384966,0,0.885638,1.228,7.469455,i,0.971,3.434885,19,0.141225,...,0,-44,1.229060,18,1.664722,8.424559,0.006545,0.002283,1,True
327614,8528779,1,0.578615,0.764,1.507408,i,0.289,1.133055,24,0.096716,...,0,1,3.936736,21,1.489439,4.443427,0.009498,-0.011970,0,True
327843,11326222,0,0.397622,1.557,1.052445,r,0.851,0.499847,15,0.128314,...,0,10,1.441788,8,4.121043,9.206562,0.004271,0.003226,1,True
401976,10593439,0,0.889502,1.063,2.600672,i,0.653,2.453330,10,0.071020,...,0,-29,0.554197,23,1.140166,8.073249,0.005943,-0.010684,1,True
430110,11242371,1,0.825296,0.505,4.939964,r,0.494,3.912424,20,0.091339,...,1,-45,0.720951,29,0.902432,3.871814,0.016204,-0.049339,0,True
512022,10866185,1,0.688360,1.774,3.134947,z,1.181,2.906849,10,0.040067,...,1,-34,0.436682,14,1.815972,9.314324,0.003607,-0.006792,0,True


In [92]:
## Save to new csv file
op_fname='new_features.csv'
df.to_csv(op_fname,index=False)